# NS spoornet

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
# we halen eerst alle lijnen op (bijv. Maatricht - Haarlem)
url = "https://www.rijdendetreinen.nl"
req = requests.get(url + "/trajecten")
soup = BeautifulSoup(req.content, 'html.parser')
trajecten = soup.select("body > div.wrap > div.container > ul")
trajecten = trajecten[0].find_all("a")
trajecten_naam = [r.text for r in trajecten]
trajecten_urls = [url + r.get("href") for r in trajecten]

In [3]:
df = pd.DataFrame(columns=["line", "station"])
for i in range(len(trajecten)):
    req = requests.get(trajecten_urls[i])
    soup_traject = BeautifulSoup(req.content, 'html.parser')
    stations = soup_traject.select(".line-stations")
    stations = stations[0].find_all("a") 
    stations_naam = [s.text for s in stations]
    stations_urls = [url + s.get("href") for s in stations]
    
    for y in range(len(stations)):
        df.loc[len(df)] = {"line":trajecten_naam[i], "station": stations_naam[y]}

df

,line,station
0,'s-Hertogenbosch - Eindhoven,'s-Hertogenbosch
1,'s-Hertogenbosch - Eindhoven,Vught
2,'s-Hertogenbosch - Eindhoven,Boxtel
3,'s-Hertogenbosch - Eindhoven,Best
4,'s-Hertogenbosch - Eindhoven,Eindhoven Strijp-S
...,...,...
799,Winterswijk - Zutphen,Winterswijk West
800,Winterswijk - Zutphen,Lichtenvoorde-Groenlo
801,Winterswijk - Zutphen,Ruurlo
802,Winterswijk - Zutphen,Vorden


In [4]:
# we gaan nu het stationsmaster data bestand inlezen (te downloaden via de website rijdendetreinen.nl)
mdStations = pd.read_csv("data/stations-2020-01-nl.csv")
mdStations.head(100)

,id,code,uic,name_short,name_medium,name_long,slug,country,type,geo_lat,geo_lng
0,266,HT,8400319,Den Bosch,'s-Hertogenbosch,'s-Hertogenbosch,s-hertogenbosch,NL,knooppuntIntercitystation,51.690480,5.293620
1,269,HTO,8400320,Dn Bosch O,'s-Hertogenb. O.,'s-Hertogenbosch Oost,s-hertogenbosch-oost,NL,stoptreinstation,51.700554,5.318333
2,227,HDE,8400388,'t Harde,'t Harde,'t Harde,t-harde,NL,stoptreinstation,52.409168,5.893611
3,51,ATN,8400045,Aalten,Aalten,Aalten,aalten,NL,stoptreinstation,51.921327,6.578627
4,5,AC,8400047,Abcoude,Abcoude,Abcoude,abcoude,NL,stoptreinstation,52.278500,4.977000
...,...,...,...,...,...,...,...,...,...,...,...
95,214,GVC,8400282,Den Haag C,Den Haag C.,Den Haag Centraal,den-haag-centraal,NL,megastation,52.080276,4.325000
96,213,GV,8400280,Dn Haag HS,Den Haag HS,Den Haag HS,den-haag-hs,NL,knooppuntIntercitystation,52.069721,4.322500
97,302,LAA,8400380,Laan v NOI,Laan v NOI,Den Haag Laan v NOI,den-haag-laan-v-noi,NL,knooppuntStoptreinstation,52.078609,4.342778
98,215,GVM,8400278,Mariahoeve,Mariahoeve,Den Haag Mariahoeve,den-haag-mariahoeve,NL,stoptreinstation,52.090557,4.369444


In [5]:
# we gaan nu de stationsgegevens erbij zetten
data = pd.merge(
    df                               # df waaraan de gegevens moeten worden toegevoegd
    ,mdStations                      # df van waaruit de gegevens komen
    ,how='left'                      # type join (left, inner, ...)
    ,left_on='station'               # kolom in df1 waarmee gejoined moet worden
    ,right_on='name_long'            # kolom in df2 waarmee gejoined moet worden
)
data

,line,station,id,code,uic,name_short,name_medium,name_long,slug,country,type,geo_lat,geo_lng
0,'s-Hertogenbosch - Eindhoven,'s-Hertogenbosch,266.0,HT,8400319.0,Den Bosch,'s-Hertogenbosch,'s-Hertogenbosch,s-hertogenbosch,NL,knooppuntIntercitystation,51.690480,5.293620
1,'s-Hertogenbosch - Eindhoven,Vught,482.0,VG,8400667.0,Vught,Vught,Vught,vught,NL,stoptreinstation,51.655556,5.291945
2,'s-Hertogenbosch - Eindhoven,Boxtel,109.0,BTL,8400129.0,Boxtel,Boxtel,Boxtel,boxtel,NL,knooppuntStoptreinstation,51.584330,5.319120
3,'s-Hertogenbosch - Eindhoven,Best,73.0,BET,8400112.0,Best,Best,Best,best,NL,stoptreinstation,51.509998,5.389167
4,'s-Hertogenbosch - Eindhoven,Eindhoven Strijp-S,784.0,EHS,8400196.0,Strijp-S,Strijp-S,Eindhoven Strijp-S,eindhoven-strijp-s,NL,stoptreinstation,51.451111,5.455833
...,...,...,...,...,...,...,...,...,...,...,...,...,...
799,Winterswijk - Zutphen,Winterswijk West,523.0,WWW,8400700.0,Wintersw W,Winterswijk W.,Winterswijk West,winterswijk-west,NL,stoptreinstation,51.974530,6.704170
800,Winterswijk - Zutphen,Lichtenvoorde-Groenlo,316.0,LTV,8400395.0,Lichtenv-G,Lichtenvoorde-G.,Lichtenvoorde-Groenlo,lichtenvoorde-groenlo,NL,stoptreinstation,52.011880,6.596410
801,Winterswijk - Zutphen,Ruurlo,400.0,RL,8400537.0,Ruurlo,Ruurlo,Ruurlo,ruurlo,NL,stoptreinstation,52.081089,6.449266
802,Winterswijk - Zutphen,Vorden,476.0,VD,8400661.0,Vorden,Vorden,Vorden,vorden,NL,stoptreinstation,52.107195,6.317046


In [8]:
# sla het bestand op als csv
data.to_csv('data/NLtrains.csv')

# NB: het is nodig om ook de index mee te exporteren aangezien de volgorde van de stations in het traject ook belangrijk is

In [59]:
# nu gaan we van alle stations de foto's downloaden
from PIL import Image
url = "https://nl.wikipedia.org/wiki/Special:Search/"

for i in range(len(mdStations)):
    try:
        req = requests.get(url+'Station%20'+mdStations['name_long'][i])
        soup = BeautifulSoup(req.content, 'html.parser')
        image = soup.select('table.infobox img')
        if image:                                                       # in case Wikipedia contains a website
            image_url = 'http:'+image[0]['src']
            img = Image.open(requests.get(image_url, stream = True).raw)
            img.save('data/pictures/'+mdStations['slug'][i]+'.jpg')
            print('Image saved: {}'.format(mdStations['slug'][i]+'.jpg'))
    except:
        print('Failed to save image: {}'.format(mdStations['slug'][i]+'.jpg'))

Image saved: s-hertogenbosch.jpg
Image saved: s-hertogenbosch-oost.jpg
Image saved: t-harde.jpg
Image saved: aalten.jpg
Image saved: abcoude.jpg
Image saved: akkrum.jpg
Image saved: alkmaar.jpg
Image saved: alkmaar-noord.jpg
Image saved: almelo.jpg
Image saved: almelo-de-riet.jpg
Image saved: almere-buiten.jpg
Image saved: almere-centrum.jpg
Image saved: almere-muziekwijk.jpg
Image saved: almere-oostvaarders.jpg
Image saved: almere-parkwijk.jpg
Image saved: almere-poort.jpg
Image saved: alphen-a-d-rijn.jpg
Image saved: amersfoort-centraal.jpg
Image saved: amersfoort-schothorst.jpg
Image saved: amersfoort-vathorst.jpg
Image saved: amsterdam-amstel.jpg
Image saved: amsterdam-bijlmer-arena.jpg
Image saved: amsterdam-centraal.jpg
Image saved: amsterdam-holendrecht.jpg
Image saved: amsterdam-lelylaan.jpg
Image saved: amsterdam-muiderpoort.jpg
Image saved: amsterdam-rai.jpg
Image saved: amsterdam-science-park.jpg
Image saved: amsterdam-sloterdijk.jpg
Image saved: amsterdam-zuid.jpg
Image sav